<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.3: Control Flow
**Prev: [Combinational Logic](2.2_comb_logic.ipynb)**<br>
**Next: [Sequential Logic](2.4_sequential_logic.ipynb)**

## Motivation
到目前为止，在 Chisel 中软件和硬件之间存在很强的对应关系。
在控制流中，我们对两者的思考方式会有更大的差异。
本模块介绍了生成器软件和硬件中的控制流。
如果重新连接到一个 Chisel 线会发生什么？
如何制作一个有多个输入的选择器（mux）？
通过完成本模块，这些问题的答案及更多内容将属于你。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Last Connect Semantics

<span style="color:blue">**Example: Reassignment**</span><br>
如前所见，Chisel 允许你使用 `:=` 操作符连接组件。
由于各种原因，可能会对同一组件发出多个连接语句。
当这种情况发生时，最后一个语句胜出。

In [ ]:
class LastConnect extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := 1.U
  io.out := 2.U
  io.out := 3.U
  io.out := 4.U
}

//  Test LastConnect
test(new LastConnect) { c => c.io.out.expect(4.U) } // Assert that the output correctly has 4
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

---
# `when`, `elsewhen`, and `otherwise`
Chisel 的主要条件逻辑实现是 `when`、`elsewhen` 和 `otherwise` 结构。
这通常看起来像这样：
```scala
when(someBooleanCondition) {
  // 当条件为真时执行的操作
}.elsewhen(someOtherBooleanCondition) {
  // 在此条件下执行的操作
}.otherwise {
  // 当所有条件都不为真时执行的操作
}
```
它们必须以上述顺序出现，尽管后两者可以省略。
可以根据需要添加任意数量的 elsewhen 子句。
任何为真的部分都会终止该结构。
三个部分中的操作可以是复杂的代码块，并且可以包含嵌套的 `when` 和类似结构。
**不同于** Scala 的 `if`，`when` 相关的代码块不会返回值。
不能这样写：
```scala
val result = when(squareIt) { x * x }.otherwise { x }
```
这将**不起作用**。我们将在 *Wires* 部分讨论解决方案。

<span style="color:blue">**Example: Chisel Conditionals**</span><br>
下面是一个使用 `when` 结构的 `Module` 示例。

In [ ]:
// Max3 returns the max of its 3 arguments
class Max3 extends Module {
  val io = IO(new Bundle {
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out = Output(UInt(16.W))
  })
    
  when(io.in1 >= io.in2 && io.in1 >= io.in3) {
    io.out := io.in1  
  }.elsewhen(io.in2 >= io.in3) {
    io.out := io.in2 
  }.otherwise {
    io.out := io.in3
  }
}

// Test Max3
test(new Max3) { c =>
  // verify that the max of the three inputs is correct
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(2.U)
  c.io.out.expect(6.U)  // input 1 should be biggest
  c.io.in2.poke(7.U)
  c.io.out.expect(7.U)  // now input 2 is
  c.io.in3.poke(11.U)
  c.io.out.expect(11.U) // and now input 3
  c.io.in3.poke(3.U)
  c.io.out.expect(7.U)  // show that decreasing an input works as well
  c.io.in1.poke(9.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(6.U)
  c.io.out.expect(9.U)  // still get max with tie
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

---
# The `Wire` Construct
让我们回到上面提到的关于 `when` 不返回值的限制。Chisel 的 `Wire` 结构是解决此问题的方法之一。
`Wire` 定义了一个电路组件，它可以出现在连接 `:=` 操作符的右侧或左侧。

<span style="color:blue">**Example: 4-Input Sort with Wires**</span><br>
为了说明这一点，让我们制作一个小型组合排序器，将其四个数字输入排序为四个数字输出。为了更清晰，考虑以下图表。当左侧值小于右侧值时，数据沿红线流动；当左侧值大于右侧值时，数据沿黑线流动，交换值。
![Sort4](images/Sorter4.png)
该图显示了一系列名称以 *row* 开头的单元格，我们将使用 `Wire`s 来构建这些单元格，其中放置特定复制或交换的结果。这段代码相当冗长，但稍后你会看到缩短它的方法。

In [ ]:
/** Sort4 sorts its 4 inputs to its 4 outputs */
class Sort4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(io.in0 < io.in1) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(io.in2 < io.in3) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(row11 < row12) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  val row20 = Wire(UInt(16.W))
  val row23 = Wire(UInt(16.W))
  when(row10 < row13) {
    row20 := row10            // preserve middle 2 elements
    row23 := row13
  }.otherwise {
    row20 := row13            // swap middle two elements
    row23 := row10
  }

  when(row20 < row21) {
    io.out0 := row20            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row20
  }

  when(row22 < row23) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row23
  }.otherwise {
    io.out2 := row23            // swap first two elements
    io.out3 := row22
  }
}


// Here's the tester
test(new Sort4) { c =>
  // verify the inputs are sorted
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(3.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(9.U)
  c.io.out3.expect(12.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


这是一个更详尽的测试器，使用了一些 Scala 的 `List` 功能。你会在后续模块中看到更多的 `List` 函数。

In [ ]:
// Here's the tester
test(new Sort4) { c =>
  // verify the all possible ordering of 4 numbers are sorted
  List(1, 2, 3, 4).permutations.foreach { case i0 :: i1 :: i2 :: i3 :: Nil =>
    println(s"Sorting $i0 $i1 $i2 $i3")
    c.io.in0.poke(i0.U)
    c.io.in1.poke(i1.U)
    c.io.in2.poke(i2.U)
    c.io.in3.poke(i3.U)
    c.io.out0.expect(1.U)
    c.io.out1.expect(2.U)
    c.io.out2.expect(3.U)
    c.io.out3.expect(4.U)
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

---
# Exercises

<span style="color:red">**Exercise: Polynomial Circuit**</span><br>
构建一个 `Module` 来计算以下多项式的值。
- $x^2 - 2x + 1$
- $2x^2 + 6x + 3$
- $4x^2 - 10x - 5$

一个选择器输入将决定计算哪个多项式。使用 `Wire`，使得 $x^2$ 的计算在模块中只出现一次，并且只有一个连接到输出。

首先，让我们使用测试驱动开发并使用 Scala 编写一个模型。完成这些函数定义以通过以下断言。这不是详尽的检查，而是合理性检查。

In [ ]:
def poly0(x: Int): Int = ???
def poly1(x: Int): Int = ???
def poly2(x: Int): Int = ???

assert(poly0(0) == 1)
assert(poly1(0) == 3)
assert(poly2(0) == -5)

assert(poly0(1) == 0)
assert(poly1(1) == 11)
assert(poly2(1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly0(x: Int): Int = x*x - 2*x + 1
def poly1(x: Int): Int = 2*x*x + 6*x + 3
def poly2(x: Int): Int = 4*x*x - 10*x - 5
</pre></article></div></section></div>

To make it even easier let's make a function that works like our desired hardware module. Use Scala `if` statements to select the polynomial based on the `select` input.

In [ ]:
def poly(select: Int, x: Int): Int = {
  ???
}

assert(poly(1, 0) == 3)
assert(poly(1, 1) == 11)
assert(poly(2, 1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly(select: Int, x: Int): Int = {
  if(select == 0) {
    poly0(x)
  }
  else if(select == 1) {
    poly1(x)
  }
  else {
    poly2(x)
  }
}
</pre></article></div></section></div>

Looks like the values are correct. So now use the following template to implement your circuit.

In [ ]:
// compute the polynomial
class Polynomial extends Module {
  val io = IO(new Bundle {
    val select = Input(UInt(2.W))
    val x = Input(SInt(32.W))
    val fOfX = Output(SInt(32.W))
  })
    
  val result = Wire(SInt(32.W))  
  val square = Wire(SInt(32.W))  
  
  ???

  io.fOfX := result  
}

// Test Polynomial
test(new Polynomial) { c =>
  for(x <- 0 to 20) {
    for(select <- 0 to 2) {
      c.io.select.poke(select.U)
      c.io.x.poke(x.S)
      c.io.fOfX.expect(poly(select, x).S)
    }
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  square := io.x * io.x
  when(io.select === 0.U) {
    result := (square - (2.S * io.x)) + 1.S
  }.elsewhen(io.select === 1.U) {
    result := (2.S * square) + (6.S * io.x) + 3.S
  }.otherwise {
    result := (4.S * square) - (10.S * io.x) - 5.S
  }
</pre></article></div></section></div>

<span style="color:red">**Exercise: Finite State Machine**</span><br>
使用卡诺图来优化状态机的逻辑既繁琐又乏味，综合工具可以解决这个问题。但它也会生成不直观且难以阅读的代码。因此，我们将使用 Chisel 控制流和最后连接语义来编写一个更合理的状态机。

研究生在其职业生涯中经历四个状态：Idle、Coding、Writing 和 Graduating。这些状态基于三个输入进行转换：咖啡、他们想到的点子以及导师施加的压力。一旦他们毕业（Graduating），他们就会回到 Idle 状态。下面的 FSM 图显示了这些状态和转换。任何未标记的转换（即没有输入时）会使研究生返回 Idle 状态，而不是停留在当前状态。输入优先级是咖啡 > 点子 > 压力，因此当处于 Idle 状态并同时收到咖啡和压力时，研究生将进入 Coding 状态。

<img src="images/fsm.png" width="500" />

首先，我们将构建一个模型来测试我们的硬件。完成以下对状态机的功能描述。它有四个输入。输出是下一个状态。状态映射已经为你提供。你可以像 `states("grad")` 这样访问它。

In [ ]:
// state map
def states = Map("idle" -> 0, "coding" -> 1, "writing" -> 2, "grad" -> 3)

// life is full of question marks
def gradLife (state: Int, coffee: Boolean, idea: Boolean, pressure: Boolean): Int = {
  var nextState = states("idle")
  ???
  nextState
}

// some sanity checks
(0 until states.size).foreach{ state => assert(gradLife(state, false, false, false) == states("idle")) }
assert(gradLife(states("writing"), true, false, true) == states("writing"))
assert(gradLife(states("idle"), true, true, true) == states("coding"))
assert(gradLife(states("idle"), false, true, true) == states("idle"))
assert(gradLife(states("grad"), false, false, false) == states("idle"))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  if (state == states("idle")) {
    if      (coffee) { nextState = states("coding") }
    else if (idea) { nextState = states("idle") }
    else if (pressure) { nextState = states("writing") }
  } else if (state == states("coding")) {
    if      (coffee) { nextState = states("coding") } 
    else if (idea || pressure) { nextState = states("writing") }
  } else if (state == states("writing")) {
    if      (coffee || idea) { nextState = states("writing") }
    else if (pressure) { nextState = states("grad") }
  }
</pre></article></div></section></div>

由于你还没有学习顺序逻辑，当前状态是 `Module` 的一个输入，下一个状态是一个输出，就像之前的 `gradLife` 函数一样。现在用 Chisel 实现状态机以通过测试。Chisel 为我们提供了一个方便的状态机映射函数，称为 `Enum`。要使用这些状态，请将它们视为 `UInt` 字面量。记住，硬件等式使用三个等号！

In [ ]:
// life gets hard-er
class GradLife extends Module {
  val io = IO(new Bundle {
    val state = Input(UInt(2.W))
    val coffee = Input(Bool())
    val idea = Input(Bool())
    val pressure = Input(Bool())
    val nextState = Output(UInt(2.W))
  })
    
  val idle :: coding :: writing :: grad :: Nil = Enum(4)
  
  ???
}


// Test
test(new GradLife) { c =>
  // verify that the hardware matches the golden model
  for (state <- 0 to 3) {
    for (coffee <- List(true, false)) {
      for (idea <- List(true, false)) {
        for (pressure <- List(true, false)) {
          c.io.state.poke(state.U)
          c.io.coffee.poke(coffee.B)
          c.io.idea.poke(idea.B)
          c.io.pressure.poke(pressure.B)
          c.io.nextState.expect(gradLife(state, coffee, idea, pressure).U)
        }
      }
    }
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-5" />
<label for="check-5"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.nextState := idle
  when (io.state === idle) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea) { io.nextState := idle }
    .elsewhen (io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === coding) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea || io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === writing) {
    when      (io.coffee || io.idea) { io.nextState := writing }
    .elsewhen (io.pressure) { io.nextState := grad }
  }
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)